In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [30]:
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [31]:
%config IPCompleter.greedy=True

In [32]:
#Load in the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [33]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 


#Count frequency of each value
Y_train.value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [34]:
# Normalize the data
X_train = X_train / 255.0
test = test / 255.0

In [35]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [36]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes = 10)

In [37]:
# Set the random seed
random_seed = 1404

In [38]:
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [39]:
#Double check the shape
X_train.shape

(37800, 28, 28, 1)

## Smaller Grid Search

(Eric you can resume the code)

In [40]:
dropout_rate = [0.05,  0.2, 0.3]
batch_size = [100, 200]
epochs = [3]
optimizer = ['SGD', 'RMSprop', 'Adamax']

In [41]:
X = X_train
Y = Y_train

#Create the model we will use on the grid search
def create_model(dropout_rate=0.25, weight_constraint=0, optimizer = 'RMSProp'):
    # create model
    model = Sequential()

    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(dropout_rate))


    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(dropout_rate))
    
    #FROM TWO SECTIONS DOWN
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation = "softmax"))
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [42]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=2)
# define the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs, dropout_rate=dropout_rate, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/3
 - 135s - loss: 1.8080 - acc: 0.4715
Epoch 2/3
 - 134s - loss: 0.4216 - acc: 0.8676
Epoch 3/3
 - 134s - loss: 0.2452 - acc: 0.9249
Epoch 1/3
 - 135s - loss: 1.6760 - acc: 0.5087
Epoch 2/3
 - 134s - loss: 0.3623 - acc: 0.8881
Epoch 3/3
 - 134s - loss: 0.2238 - acc: 0.9319
Epoch 1/3
 - 134s - loss: 1.7496 - acc: 0.4437
Epoch 2/3
 - 134s - loss: 0.4682 - acc: 0.8515
Epoch 3/3
 - 134s - loss: 0.2812 - acc: 0.9121
Epoch 1/3
 - 135s - loss: 0.2609 - acc: 0.9173
Epoch 2/3
 - 134s - loss: 0.0623 - acc: 0.9813
Epoch 3/3
 - 134s - loss: 0.0426 - acc: 0.9866
Epoch 1/3
 - 135s - loss: 0.2626 - acc: 0.9173
Epoch 2/3
 - 134s - loss: 0.0583 - acc: 0.9816
Epoch 3/3
 - 134s - loss: 0.0374 - acc: 0.9887
Epoch 1/3
 - 134s - loss: 0.2356 - acc: 0.9250
Epoch 2/3
 - 134s - loss: 0.0633 - acc: 0.9810
Epoch 3/3
 - 134s - loss: 0.0400 - acc: 0.9875
Epoch 1/3
 - 135s - loss: 0.2512 - acc: 0.9188
Epoch 2/3
 - 135s - loss: 0.0715 - acc: 0.9775
Epoch 3/3
 - 135s - loss: 0.0478 - acc: 0.9844
Epoch 1/3
 - 